In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist as mnist
import numpy as np

In [2]:
mnist_data = mnist.input_data.read_data_sets("/tmp/data", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
def reshape(x):
    s = tf.reshape(x, [-1, 28, 28, 1])
    d = tf.zeros([x.get_shape().as_list()[0], 28, 27, 1], dtype=tf.float32)
    return tf.concat(2,[d, s])
learning_rate = 0.001
training_iters = 2
batch_size = 128
display_step = 10

X = tf.placeholder(tf.float32,[batch_size, 784])
X_reshaped = reshape(X)
Y = tf.placeholder(tf.float32,[1, 10])

In [4]:
def update(X, weights, b):
    # Convolution operation
    conv_z = tf.nn.conv2d(X_reshaped, weights["W_z"], strides = [1, 1, 1, 1], padding = "VALID")
    conv_z_reshaped = tf.nn.tanh(tf.transpose(conv_z, perm = [0,2,3,1]))
    conv_f = tf.nn.conv2d(X_reshaped, weights["W_f"], strides = [1, 1, 1, 1], padding = "VALID")
    conv_f_reshaped = tf.nn.sigmoid(tf.transpose(conv_f, perm = [0,2,3,1]))
    conv_o = tf.nn.conv2d(X_reshaped, weights["W_o"], strides = [1, 1, 1, 1], padding = "VALID")
    conv_o_reshaped = tf.nn.sigmoid(tf.transpose(conv_o, perm = [0,2,3,1]))
    c_conv = (1-conv_f_reshaped)*conv_z_reshaped

    # Pooling operation
    with tf.device('/gpu:0'):
        for j in range(2):
            if j == 0:
                c = weights["c"][-1,:,:,:]
            else:
                c = weights["c"][j-1,:,:,:]
            for i in range(c.get_shape()[1]):
                tmp = c[i,:,:]*conv_f_reshaped[j,i,:,:] + c_conv[j, i, :, :]
                tmp = tf.reshape(tmp,[1,tmp.get_shape().as_list()[0],tmp.get_shape().as_list()[1]])
                if i == 0:
                    c = tf.concat(0,[tmp, c[i+1:,:,:]])
                else:
                    c = tf.concat(0,[c[:i,:,:],tmp, c[i+1:,:,:]])
            c = tf.reshape(c, [1, c.get_shape().as_list()[0], c.get_shape().as_list()[1], c.get_shape().as_list()[2]])
            if j==0:
                weights["c"] = tf.concat(0,[c, weights["c"][j+1:,:,:,:]])
            else:
                weights["c"] = tf.concat(0,[weights["c"][:j,:,:,:], c, weights["c"][j+1:,:,:,:]])
            weights["h"] = conv_o_reshaped*weights["c"]

    fc1 = tf.reshape(weights["h"], [-1, 560])
    fc1 = tf.nn.tanh(tf.add(tf.matmul(fc1, weights["W"]), b))
    return fc1


In [5]:
# Number of timesteps(T in the paper) and no of filters (m in the paper) are 28 and 20 respectively
weights = {
    "W_z" : tf.Variable(tf.random_normal([28, 28, 1, 20])),
    "W_f" : tf.Variable(tf.random_normal([28, 28, 1, 20])),
    "W_o" : tf.Variable(tf.random_normal([28, 28, 1, 20])),
    "W" : tf.Variable(tf.random_normal([560, 10])),
    "h" : tf.Variable(tf.zeros([batch_size, 28, 20, 1], dtype = tf.float32)),
    "c" : tf.Variable(tf.zeros([batch_size, 28, 20, 1], dtype = tf.float32))
        
}
b = tf.Variable(tf.random_normal([10]))


In [6]:
pred = update(X_reshaped, weights, b)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [ ]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

    # Calculate accuracy for 256 mnist test images
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256]})